In [1]:
%%capture
! pip install ml-collections

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/gbvs

In [2]:
import sys
import cv2
import time
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import pickle
import random
import copy

import os
import shutil
from PIL import Image, ImageDraw

import torch
from torch.utils.data import Dataset
from torchvision.transforms.functional import to_tensor, normalize
from torchvision import transforms

from saliency_models import gbvs, ittikochneibur
from coco.dataset import COCODataset
from utils import *
sys.path.append("..")

Run on COCO18 test dataset

In [4]:
%%capture
test_annotations = "../../debug/coco18_test.json"
test_imagedir = "../../coco18_test"
image_list = os.listdir(test_imagedir)
input_images = COCODataset(test_annotations, test_imagedir, image_size=(224,224))

In [5]:
img_size = (320, 512)
tg_bbox_dic = {}

for id in trange(len(input_images)):
    _, _, bbox, labels_cpu = input_images[id]
    
    file_name_full = input_images.id2file[input_images.annotations[id]['image_id']]
    file_name = file_name_full.split('/')[-1]

    tg_loc = bbox_cordinates(bbox, img_size[1], img_size[0])
    tg_bbox_dic[file_name] = tg_loc

100%|██████████| 612/612 [00:32<00:00, 18.83it/s]


In [6]:
gbvs_res, size, bugs = [], 48, [257, 272, 447]
gbvs_attention_map, scanpath = {}, {}

for i in trange(len(input_images)):
    if i in bugs:
        continue
    # get the target bounding box
    tg_loc = tg_bbox_dic[image_list[i]]

    # get the img path
    img_dir = os.path.join(test_imagedir, image_list[i]) 
    img = cv2.imread(img_dir)

    # compute the saliency map
    saliency_map_gbvs = gbvs.compute_saliency(img)

    saliency_map_gbvs_tensor = torch.from_numpy(saliency_map_gbvs).detach()
    saliency_map_gbvs_tensor = transforms.Resize(img_size)(saliency_map_gbvs_tensor.unsqueeze(0))

    gbvs_attention_map[image_list[i]] = copy.deepcopy(saliency_map_gbvs_tensor)

    search_num, path = searchProcesswithPath(tg_loc, saliency_map_gbvs_tensor, img_size, size)
    scanpath[image_list[i]] = path
    print(search_num)
    gbvs_res.append(search_num)

  0%|          | 1/612 [00:04<40:51,  4.01s/it]

12


  0%|          | 2/612 [00:07<37:18,  3.67s/it]

2


  0%|          | 3/612 [00:11<37:37,  3.71s/it]

...

4


100%|█████████▉| 611/612 [40:58<00:03,  3.74s/it]

22


100%|██████████| 612/612 [41:02<00:00,  4.02s/it]

7


In [7]:
gbvs_res_fig5 = {}
gbvs_res_fig5['search_list'] = gbvs_res
gbvs_res_fig5['attention_map'] = gbvs_attention_map
gbvs_res_fig5['scanpath'] = scanpath

In [10]:
with open("../results/COCO/[Fig5]coco_gbvs_res.pkl", "wb") as tf:
    pickle.dump(gbvs_res_fig5, tf)

In [ ]:
gbvs_accu_coco = model_performance(gbvs_res, len(gbvs_res))
gbvs_accu_coco[:11]

[0,
 0.0,
 0.21018062397372742,
 0.3070607553366174,
 0.3842364532019704,
 0.451559934318555,
 0.49261083743842365,
 0.548440065681445,
 0.5779967159277504,
 0.6059113300492611,
 0.6305418719211823]

In [ ]:
with open("../results/COCO/coco_gbvs_accu_performance.pkl", "wb") as tf:
    pickle.dump(gbvs_accu_coco,tf)